In [ ]:
# use_inference.py

import pandas as pd
from model_initialiser import predict_latest

def main():
    # 1) load your (n_inst, T) price matrix
    prices = pd.read_csv("prices.txt", sep=r"\s+", header=None).values.T

    t = 118
    prices = prices[:, : t+1]  #Smallest is 118 for logits, 100 to fill first cache and another 20 for the next cache. -1 because first 100 contributes to the 20cache

    # 2) get back a 1-D array of length n_inst with your “right-now” regimes
    preds = predict_latest(
        prices,
        model_path = "bilstm_self2.pth",
        seq_len    = 20,
        win_len    = 100,
        device     = "cpu",
        # return_logits=False by decfault, so you get arg-max
    )

    print("Shape:", preds.shape)  # e.g. (50,)
    print("Preds :", preds)

if __name__ == "__main__":
    main()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
from model_initialiser import predict_latest

# ─────────────────────────────────────────────────────────────────────────────
# plot_regimes.py
# Plots regimes and price on the same chart for each instrument.
# ─────────────────────────────────────────────────────────────────────────────

def main():
    # Load full price matrix (n_inst, T_full)
    prices_full = pd.read_csv('prices.txt', sep=r'\s+', header=None).values.T
    n_inst, T_full = prices_full.shape

    # Define the time range and model parameters
    start_t = 118
    end_t   = 750
    seq_len = 20
    win_len = 100

    # Number of timesteps to predict
    n_ts = end_t - start_t + 1
    timesteps = np.arange(start_t, end_t + 1)

    # Allocate array to hold regime predictions: shape (n_inst, n_ts)
    preds_all = np.zeros((n_inst, n_ts), dtype=int)

    # Loop over each timestep and compute regimes
    for idx, t in enumerate(timesteps):
        prices = prices_full[:, :t+1]
        preds  = predict_latest(
            prices,
            model_path='bilstm_self2.pth',
            seq_len=seq_len,
            win_len=win_len,
            device='cpu'
        )
        preds_all[:, idx] = preds

    # Plot price + regime for each instrument
    for inst in range(n_inst):
        fig, ax = plt.subplots(figsize=(10, 4))

        # Plot price
        price_series = prices_full[inst, start_t:end_t+1]
        ax.plot(timesteps, price_series, label='Price')
        ax.set_ylabel('Price')

        # Shade regimes: bear=0 red, bull=2 green
        regime = preds_all[inst]
        y_min, y_max = ax.get_ylim()
        for regime_val, color in [(0, 'red'), (2, 'green')]:
            mask = regime == regime_val
            # find contiguous segments
            starts = np.where(np.logical_and(mask, np.concatenate([[False], mask[:-1]])))[0]
            ends   = np.where(np.logical_and(mask, np.concatenate([mask[1:], [False]])))[0]
            for s, e in zip(starts, ends):
                ax.axvspan(timesteps[s], timesteps[e], color=color, alpha=0.2)

        ax.set_title(f'Instrument {inst} Regimes and Price')
        ax.set_xlabel('Timestep')
        ax.legend(loc='upper left')
        plt.tight_layout()
        plt.show()

if __name__ == '__main__':
    main()


In [ ]:
Shape: (50,)
Preds : [2 0 0 0 2 2 0 2 2 2 2 2 0 0 2 0 2 2 2 2 0 2 2 2 2 0 2 2 0 0 0 0 2 0 2 2 2
 0 2 0 2 2 2 0 0 2 2 2 2 2]

In [ ]:
import numpy as np
from model_initialiser import predict_latest


def main():
    # ---------------------------------------------------
    # Mock historical price data (n_inst instruments, full T timesteps)
    n_inst = 50
    T_full = 200
    rng = np.random.RandomState(42)
    # prices in range [100, 200)
    full_prices = rng.rand(n_inst, T_full) * 100 + 100

    # Predictor parameters (must match your model setup)
    seq_len = 20
    win_len = 100
    model_path = "bilstm_self2.pth"
    device = "cpu"

    # The earliest t at which we have enough history:
    start_t = win_len + seq_len - 1  # e.g. 100 + 20 - 1 = 119

    # Iterate from first valid timestep up to the end,
    # each loop "expands" the 2D prices array by slicing [:, :t+1]
    for t in range(start_t, T_full):
        # slice to simulate streaming new bar
        prices_t = full_prices[:, : t+1]  # shape (n_inst, t+1)

        # call predict_latest exactly as in your use_inference script
        preds = predict_latest(
            prices_t,
            model_path = model_path,
            seq_len    = seq_len,
            win_len    = win_len,
            device     = device
        )

        # print out the time index and the returned regimes
        print(f"t = {t:3d}  -->  preds: {preds}")


if __name__ == "__main__":
    main()